# Import dependencies

In [223]:
import os
import re
import pickle
import numpy as np
import pandas as pd
import importlib

import spacy

from utilities import _support
from utilities._support import ig_profiles

from nltk.tokenize import word_tokenize, sent_tokenize

importlib.reload(_support)

<module 'utilities._support' from 'c:\\Users\\tommaso\\Documents\\_Projects\\IGScraper\\utilities\\_support.py'>

# Concat CSVs from same profile

In [38]:
csv_path = 'csv/matteosalviniofficial/'
csv_files = os.listdir(csv_path)

indagato = ig_profiles['salvini']

In [39]:
postfiles = list(set([csv for csv in csv_files if csv.startswith(indagato) and csv.endswith('posts.csv')]))
commentfiles = list(set([csv for csv in csv_files if csv.startswith(indagato) and csv.endswith('comments.csv')]))

assert(len(postfiles) == len(commentfiles))

In [40]:
df = pd.concat([pd.read_csv(csv_path+commentfn, sep='\t') for commentfn in commentfiles]).reset_index(drop=True)

In [44]:
print(f'shape: {df.shape}')
df.head()

shape: (206360, 5)


,post_id,user,comment,comment_datetime,comment_likes
0,Ck9MRyyKIT5,gatto_paoletto,"Tutti i cittadini dovrebbero saperlo praticare. Si salverebbero migliaia di vita ogni anno. Questa è civiltà, questa è la società che si evolve. Non con i bagni per chi non é sicuro del proprio genere...",2022-11-14T21:29:33.000Z,29
1,Ck9MRyyKIT5,marini_mariagrazia,"La Professionalità e tempestività di questa ragazza sono encomiabili! Bisognerebbe venissero fatti dei corsi di primo soccorso nelle scuole, sarebbero molto utili!",2022-11-14T21:58:05.000Z,13
2,Ck9MRyyKIT5,acquafreddafrancesco,Nessuna correlazione con il siero? Fatevi domande gente! Troppi casi in giro!,2022-11-16T07:00:14.000Z,1
3,Ck9MRyyKIT5,saiberg2000,Se stata l’angelo di quel l ‘uomo 👏🏻👏🏻👏🏻,2022-11-14T21:35:28.000Z,2
4,Ck9MRyyKIT5,_franz85_,Andrebbe insegnato a scuola,2022-11-15T05:08:19.000Z,1


In [48]:
df.to_csv(f'csv/{indagato}_comments_20230316_0-1100.csv', sep='\t', index=False)

# Load data

In [92]:
df = pd.read_csv('csv/matteosalviniofficial_comments_20230316_0-1100.csv', sep='\t')

# Utils

In [93]:
def lambda_pipe(element, lpipe):
    return lpipe[0](element) if len(lpipe)==1 else lambda_pipe(lpipe[0](element), lpipe[1:])

# Data preparation

## Text preprocess

- lower
- no urls
- no non-alpha chars:
    - no @ tag
    - no punctuation (except '.')
    - no emoji
    - no numbers
    - no escape chars
- no extra spaces
- no extra '.'

In [94]:
lower = lambda s: s.lower()
no_urls = lambda s: re.sub(r'https?://\S+|www.\.\S+', '', s)
no_nonalpha = lambda s: ''.join([c for c in s if c.isalpha() or c==' ' or c=='.'])
no_ellipses = lambda s: re.sub(r'\.{2,}', '. ', s)
no_extra_spaces = lambda s: ' '.join(s.split())

def prep_df(df, column_name='comment'):
    df['comment_prep'] = df[column_name].apply(lambda c: lambda_pipe(c, [lower, no_urls, no_nonalpha, no_ellipses, no_extra_spaces]))
    return df

In [95]:
df = prep_df(df)

df.head(2)

,post_id,user,comment,comment_datetime,comment_likes,comment_prep
0,Ck9MRyyKIT5,gatto_paoletto,"Tutti i cittadini dovrebbero saperlo praticare. Si salverebbero migliaia di vita ogni anno. Questa è civiltà, questa è la società che si evolve. Non con i bagni per chi non é sicuro del proprio genere...",2022-11-14T21:29:33.000Z,29,tutti i cittadini dovrebbero saperlo praticare. si salverebbero migliaia di vita ogni anno. questa è civiltà questa è la società che si evolve. non con i bagni per chi non é sicuro del proprio genere.
1,Ck9MRyyKIT5,marini_mariagrazia,"La Professionalità e tempestività di questa ragazza sono encomiabili! Bisognerebbe venissero fatti dei corsi di primo soccorso nelle scuole, sarebbero molto utili!",2022-11-14T21:58:05.000Z,13,la professionalità e tempestività di questa ragazza sono encomiabili bisognerebbe venissero fatti dei corsi di primo soccorso nelle scuole sarebbero molto utili


## Tokenization

1. Text → Sentences
2. Senteces → Words

In [117]:
tokenize_text2senteces = lambda text: [sentence[:-1] if sentence.endswith('.') else sentence for sentence in sent_tokenize(text)]
tokenize_senteces2words = lambda sentences: [word_tokenize(sentence) if type(sentence) is str and len(sentence)>0 else None for sentence in sentences]

def tokenize_df(df, column_name='comment_prep'):
    df['comment_token'] = df[column_name].apply(lambda c: lambda_pipe(c, [tokenize_text2senteces, lambda x: list(x), tokenize_senteces2words]))
    return df

In [121]:
df = tokenize_df(df)

## Lemmatization + POS tagging

Using spacy nlp model <br>
[ https://spacy.io/models/it#it_core_news_md ]

- Lemmatization
    - reduction to base form
- POS tag filtering:
    - nouns
    - verbs
    - adjectives
    - adverbs

In [190]:
spacy_nlp = spacy.load("it_core_news_md", disable=['ner', 'entity_linker', 'entity_ruler',
                                                    'textcat', 'textcat_multilabel', 'morphologizer', 
                                                    'senter', 'sentencizer', 'transformer'])

sentence2doc = lambda sentence: spacy_nlp(' '.join(sentence) if type(sentence) is list else sentence) if sentence!=None else None
lemmatize_doc = lambda doc: [t.lemma_ if t != None else None for t in doc] if doc!=None else None
postag_doc = lambda doc: [t.tag_ if t != None else None for t in doc] if doc!=None else None

def lemma_tagger_df(df, column_name='comment_token'):
    docs = df[column_name].apply(lambda c: [sentence2doc(sentence) for sentence in c])
    df['comment_lemma'] = docs.apply(lambda docs: [lemmatize_doc(sentence) for sentence in docs])
    df['comment_postag'] = docs.apply(lambda docs: [postag_doc(sentence) for sentence in docs])
    return df

c:\Users\tommaso\AppData\Local\Programs\Python\Python39\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'it_core_news_md' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [191]:
df = lemma_tagger_df(df)

# Save CSV

## Lookup users

In [261]:
with open('utilities/lookup_users.pkl', 'rb') as f:
    full_lookup = pickle.load(f)

this_users = set(df.user)
new_users = set(full_lookup.keys()) - this_users

In [262]:
start_idx = len(full_lookup)
for i,new_user in enumerate(new_users):
    full_lookup[new_users] = f'user_{str(start_idx+i)}'

In [263]:
with open('utilities/lookup_users.pkl', 'wb') as f:
    pickle.dump(full_lookup, f)

In [265]:
df.user = df.user.apply(lambda u: full_lookup[u])

## Write CSV as pickle

In [271]:
fname = 'gitshared_csv/matteosalviniofficial_comments_20230316_0-1100_preprocessed.df'

In [272]:
df.to_pickle(fname)